# Tech Recon Web Interface

This notebook provides a web interface for the Tech Recon project using Jupyter widgets and real-time updates.

In [1]:
# Install required packages if not already installed
!pip install ipywidgets flask flask-socketio python-socketio -q

## Option 1: Open Web App in New Tab

Run the cell below to start the web server, then click the link that appears.

In [2]:
import subprocess
import time
import os
from IPython.display import display, HTML, IFrame

# Start the web server in background
print("Starting web server...")
proc = subprocess.Popen(
    ['.venv/bin/python', 'web_app.py', '--port', '8050'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for server to start
time.sleep(3)

# Get the current JupyterLab URL
jupyter_base = os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')

# Display clickable link
display(HTML(f'''
<div style="padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
     border-radius: 10px; color: white; text-align: center;">
    <h2>🚀 Tech Recon Web Interface</h2>
    <p style="font-size: 16px;">The web server is running on port 8050</p>
    <p style="font-size: 14px; margin-top: 20px;">
        <strong>Access Methods:</strong><br/><br/>
        1. <a href="/proxy/8050/" target="_blank" 
           style="color: #ffd700; text-decoration: underline;">
           Click here to open in new tab (Method 1)
        </a><br/><br/>
        2. <a href="/proxy/absolute/8050/" target="_blank" 
           style="color: #ffd700; text-decoration: underline;">
           Click here to open in new tab (Method 2)
        </a><br/><br/>
        3. Or manually navigate to: <code style="background: rgba(0,0,0,0.3); padding: 5px;">/proxy/8050/</code>
    </p>
    <p style="font-size: 12px; margin-top: 20px; opacity: 0.8;">
        Note: If the links don't work, try the iframe option below
    </p>
</div>
'''))

print("\nServer process ID:", proc.pid)
print("To stop the server, run: kill", proc.pid)

Starting web server...



Server process ID: 1149
To stop the server, run: kill 1149


## Option 2: Display Web App in Notebook (IFrame)

Run this cell to display the web interface directly in the notebook.

In [3]:
from IPython.display import IFrame

# Display the web app in an iframe
IFrame(src='/proxy/8050/', width='100%', height=800)

## Option 3: Simple Jupyter Widget Interface

A simplified interface using Jupyter widgets (no Flask required).

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio
import sys
from datetime import datetime

# Import the main execution function
sys.path.insert(0, '.')
from main import graph_streaming_execution

# Create UI components
output_log = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'overflow_y': 'auto'})
status_label = widgets.HTML(value='<b>Status:</b> Ready')

part1_btn = widgets.Button(
    description='Run Part1',
    button_style='primary',
    tooltip='Execute Part1',
    icon='play'
)

part2_btn = widgets.Button(
    description='Run Part2',
    button_style='success',
    tooltip='Execute Part2',
    icon='play'
)

clear_btn = widgets.Button(
    description='Clear Log',
    button_style='warning',
    tooltip='Clear the log output',
    icon='trash'
)

# Execution function
async def run_execution(part):
    with output_log:
        status_label.value = f'<b>Status:</b> <span style="color: green;">Running {part}...</span>'
        part1_btn.disabled = True
        part2_btn.disabled = True
        
        try:
            print(f"\n{'='*60}")
            print(f"Starting {part} Execution")
            print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"{'='*60}\n")
            
            payload = {"user_query": part}
            
            async for event in graph_streaming_execution(payload):
                if event:
                    event_type = event.get('event_type', 'unknown')
                    
                    if event_type == 'text_chunk':
                        print(event.get('data', ''), end='', flush=True)
                    elif event_type == 'reasoning':
                        print(f"\n[REASONING] {event.get('reasoning_text', '')}")
                    elif event_type == 'tool_use':
                        print(f"\n[TOOL] {event.get('tool_name', 'unknown')}")
                    elif event_type == 'tool_result':
                        output = event.get('output', '')[:200]
                        print(f"\n[RESULT] {output}...")
                
                await asyncio.sleep(0)
            
            print(f"\n\n{'='*60}")
            print(f"{part} Execution Completed!")
            print(f"{'='*60}\n")
            
            status_label.value = '<b>Status:</b> <span style="color: blue;">Completed</span>'
            
        except Exception as e:
            print(f"\n\n❌ Error: {str(e)}")
            status_label.value = f'<b>Status:</b> <span style="color: red;">Error: {str(e)}</span>'
        
        finally:
            part1_btn.disabled = False
            part2_btn.disabled = False

def on_part1_click(b):
    asyncio.create_task(run_execution('Part1'))

def on_part2_click(b):
    asyncio.create_task(run_execution('Part2'))

def on_clear_click(b):
    output_log.clear_output()
    status_label.value = '<b>Status:</b> Ready'

part1_btn.on_click(on_part1_click)
part2_btn.on_click(on_part2_click)
clear_btn.on_click(on_clear_click)

# Layout
header = widgets.HTML(
    value='''
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
         padding: 20px; border-radius: 10px; color: white; text-align: center;">
        <h1>🚀 Tech Recon - Jupyter Interface</h1>
        <p>Real-time monitoring and execution</p>
    </div>
    '''
)

button_box = widgets.HBox([part1_btn, part2_btn, clear_btn])
status_box = widgets.VBox([status_label])

# Display the interface
display(widgets.VBox([
    header,
    widgets.HTML('<br>'),
    status_box,
    button_box,
    widgets.HTML('<h3>Execution Log:</h3>'),
    output_log
]))

print("\n✅ Jupyter interface ready! Click the buttons above to execute.")

ModuleNotFoundError: No module named 'strands'

## Stop the Web Server

Run this cell to stop the Flask web server if it's running.

In [ ]:
!pkill -f "web_app.py"
print("Web server stopped.")

## Download Generated Files

List and download files generated by the execution.

In [ ]:
import os
from pathlib import Path
import zipfile
from IPython.display import FileLink

artifacts_path = Path('./artifacts')

if artifacts_path.exists():
    print("📁 Generated Files:\n")
    
    for part in ['part1', 'part2']:
        part_path = artifacts_path / part
        if part_path.exists():
            print(f"\n{part.upper()}:")
            print("-" * 60)
            
            for file_path in part_path.rglob('*'):
                if file_path.is_file():
                    size_kb = file_path.stat().st_size / 1024
                    rel_path = file_path.relative_to(artifacts_path)
                    print(f"  📄 {file_path.name} ({size_kb:.2f} KB)")
                    display(FileLink(str(file_path), result_html_prefix=f"      → Download: "))
    
    # Create ZIP file for download
    zip_path = 'artifacts_download.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_path in artifacts_path.rglob('*'):
            if file_path.is_file():
                arcname = file_path.relative_to(artifacts_path)
                zf.write(file_path, arcname)
    
    print(f"\n\n📦 All files packaged:")
    display(FileLink(zip_path, result_html_prefix="Download all files as ZIP: "))
    
else:
    print("No artifacts found. Run Part1 or Part2 first.")